# Задание

**Контекст**

Вы — аналитик интернет-магазина. Перед вами стоит задача построить для своих коллег дашборд, по которому наглядно можно мониторить показатели интернет-магазина.

**Описание задания**

Задание 1. Постройте столбчатую диаграмму распределения количества строк в файле sales.csv в каждом регионе. Подберите параметры визуализации так, чтобы график легко читался.

Задание 2. Создайте дата фрейм sales_region, в котором для каждого региона и месяца посчитана сумма продаж. Постройте график динамики продаж по месяцам с возможностью выбора региона.

In [55]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px

In [56]:
sales = pd.read_csv('data/sales.csv')
sales.head()

,order_id,date,customer_id,category,region,sales,quantity,discount
0,59570135,2017-10-16,512,Одежда и обувь,Ставропольский край,1301,2,0.00
1,16578697,2017-07-18,512,Одежда и обувь,Ставропольский край,3530,4,0.05
2,46943601,2017-07-05,512,Бытовая техника,Ставропольский край,5773,1,0.10
3,42436035,2017-02-18,512,Спорт и отдых,Ставропольский край,4723,3,0.03
4,49798299,2017-09-08,512,Красота,Ставропольский край,1936,3,0.00


## Задание 1: количество строк по регионам

In [57]:
# Группируем данные по региону
data_by_region = sales.groupby('region')['sales'].sum()
data_by_region

region
Амурская область         901353711
Владимирская область     368734758
Иркутская область        537022358
Краснодарский край       542006190
Красноярский край        366089667
Ленинградская область    714842726
Московская область       894264890
Приморский край          546617010
Смоленская область       357781723
Ставропольский край      448592821
Хабаровский край         715289191
Name: sales, dtype: int64

In [58]:
# Используем plotly
fig = px.bar(
        data_by_region, 
        x=data_by_region.index, 
        y=data_by_region.values,
        title='Продажи',
        subtitle='По регионам'
    ).update_layout(
        xaxis_title='Регион', 
        yaxis_title='Продажи'
    )
fig.show()

## Задание 2: динамика продаж по месяцам

In [59]:
# Получим месяц и год заказа
sales['month'] = sales['date'].str.slice(start=0, stop=7)
sales

,order_id,date,customer_id,category,region,sales,quantity,discount,month
0,59570135,2017-10-16,512,Одежда и обувь,Ставропольский край,1301,2,0.00,2017-10
1,16578697,2017-07-18,512,Одежда и обувь,Ставропольский край,3530,4,0.05,2017-07
2,46943601,2017-07-05,512,Бытовая техника,Ставропольский край,5773,1,0.10,2017-07
3,42436035,2017-02-18,512,Спорт и отдых,Ставропольский край,4723,3,0.03,2017-02
4,49798299,2017-09-08,512,Красота,Ставропольский край,1936,3,0.00,2017-09
...,...,...,...,...,...,...,...,...,...
1554187,57005688,2019-07-12,100511,Спорт и отдых,Хабаровский край,10749,3,0.00,2019-07
1554188,30229596,2019-10-11,100511,Продукты питания,Хабаровский край,2205,2,0.05,2019-10
1554189,68823901,2019-05-08,100511,Дача и сад,Хабаровский край,2949,3,0.03,2019-05
1554190,99959071,2019-05-02,100511,Бытовая техника,Хабаровский край,805,3,0.05,2019-05


In [60]:
# Сгруппируем по нужным полям
sales_by_month = sales.groupby(['month', 'region']).agg({'sales': 'sum'}).reset_index()
sales_by_month

,month,region,sales
0,2017-01,Амурская область,13892593
1,2017-01,Владимирская область,5268105
2,2017-01,Иркутская область,8489321
3,2017-01,Краснодарский край,8384608
4,2017-01,Красноярский край,5333002
...,...,...,...
369,2019-10,Московская область,18560872
370,2019-10,Приморский край,10595997
371,2019-10,Смоленская область,7390256
372,2019-10,Ставропольский край,9308110


In [61]:
# Динамика продаж с возможностью выбора региона
#sales_region = sales_by_month.groupby(['month', 'region'], as_index=False).sum()
fig = px.line(sales_by_month, x='month', y='sales', color='region')

fig.update_layout(
    updatemenus=[
        {
            "buttons": [{ "label": 'Все регионы', "method": "restyle", "args": [{"visible": [True for _ in fig.data]}], }] +
            [
                {
                    "label": t.name,
                    "method": "restyle",
                    "args": [{"visible": [t2.name == t.name for t2 in fig.data]}],
                }
                for t in fig.data
            ]
        }
    ]
)

fig.show()